**Install Packages**

In [3]:
!pip install torch trochvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement trochvision (from versions: none)
ERROR: No matching distribution found for trochvision


In [4]:
!pip install laserembeddings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 KB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 5.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=d4173e89ac45fd50521e25a6978a30b5612538aae23d62f54ea42ff0827da6d6
  Stored in directory: /root/.cache/pip/wheels/c4/df/30/3d6c623db99d503dcdbae1f686953b7c1a0754d8a658dc0845
Successfully built sacremoses


In [5]:
import pandas as pd
import random as rd
import json
import numpy as np
import re

In [6]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


# Monolingual Embedding

In [7]:
from laserembeddings import Laser

laser = Laser()

# if all sentences are in the same language:

embeddings = laser.embed_sentences(
    ['به ملازمان سلطان که رساند این دعا را که به شکر پادشاهی ز نظر مران گدا را ز رقیب دیوسیرت به خدای خود پناهم مگر آن شهاب ثاقب مددی دهد خدا را',
     'هزار و سیصدتا سلام'],
    lang='fa')  # lang is only used for tokenization

In [8]:
print(embeddings.shape)
print(embeddings)
%cd "/content/"

(2, 1024)
[[ 0.01072003  0.00345858  0.00174004 ...  0.02418301  0.01499301
   0.01545107]
 [ 0.0038445   0.00011095  0.00274064 ...  0.0077541   0.00159712
  -0.00810177]]
/content


# Multilingual Embedding

In [9]:
#Example
embeddings = laser.embed_sentences(
    ['I love pasta.',
     "J'adore les pâtes.",
     'Ich liebe Pasta.'],
    lang=['en', 'fr', 'de'])

**Input data set**

read input data set and creat the input table

In [17]:
data_set_root = "/content/drive/MyDrive/Text_enrich/datasets/"
input_data = pd.read_csv(data_set_root + "input_text/input_text.csv")

In [18]:
columns = ["input_text" , "vector"]
input_table = pd.DataFrame([], columns=columns)
for sample in rd.sample(list(pd.unique(input_data["input_text"])),100):
  input_table = input_table.append({'input_text':sample,'vector':laser.embed_sentences(sample, 'fa')}, ignore_index = True)

In [19]:
input_table

,input_text,vector
0,کارها باید با مشارکت انجام شوند,"[[0.0012764974, -1.8941742e-05, 0.00084856804,..."
1,نباید غیبت دیگران را بکنیم.,"[[0.031145107, -1.6177339e-05, 0.0006270574, 0..."
2,باید به افرادی که بدهی دارند مهلت بیشتری داد,"[[0.00917535, -0.00053723203, 0.006148569, 0.0..."
3,باید پنهانی کمک‌های خود را به فقرا برسانیم,"[[0.020198734, -0.00017238935, 0.0005591394, 0..."
4,نباید در خوردن زیاده‌روی کرد,"[[0.011338596, -9.024682e-05, 0.012490797, 0.0..."
...,...,...
95,نباید براساس ظاهر افراد را قضاوت کرد,"[[0.0054349084, -0.00022610718, -0.0007558833,..."
96,باید اهل یادگیری و دانش باشیم,"[[0.0013720348, 0.00016718791, -0.00057697005,..."
97,باید فرزندان خود را به خوبی تربیت کنیم,"[[0.0029088948, -0.00011010115, 0.002297773, 0..."
98,عادت طمع انسان را پست می‌کند.,"[[0.001958303, -3.6852805e-05, 0.00015389822, ..."


# Output data set embedding




English qoutes

In [34]:
columns = ["output_text" , "vector"]
English_qoute_table = pd.DataFrame([], columns=columns)
f = open(data_set_root +'quotes/quotes.jsonl')
quotes = [json.loads(jline) for jline in f.read().splitlines()]
for i in range(len(quotes)):
  if len(quotes[i]["quote"]) < 210:
    English_qoute_table = English_qoute_table.append({'output_text':quotes[i]["quote"][1:-2],'vector':laser.embed_sentences(quotes[i]["quote"][1:-2], 'en')}, ignore_index = True)

f.close()

In [35]:
English_qoute_table

,output_text,vector
0,Be yourself; everyone else is already taken,"[[0.03283381, 0.0025794504, 0.022685742, 0.008..."
1,"I'm selfish, impatient and a little insecure. ...","[[0.018681081, 0.00092175644, 0.0030920352, 0...."
2,Two things are infinite: the universe and huma...,"[[0.005447657, -3.156499e-05, 0.0062513533, 0...."
3,"So many books, so little time","[[0.008000842, 0.0071800463, 0.0014852672, 0.0..."
4,A room without books is like a body without a ...,"[[0.0024867677, -0.00081185653, -0.0034946627,..."
...,...,...
2042,"Stepping onto a brand-new path is difficult, b...","[[0.0055467826, -0.00013396252, 0.00923926, 0...."
2043,Morality is simply the attitude we adopt towar...,"[[0.0024499279, -0.00011392398, 0.011446702, 0..."
2044,"In life, finding a voice is speaking and livin...","[[0.018904364, 0.013504211, -0.0012334686, 0.0..."
2045,"Winter is the time for comfort, for good food ...","[[0.011106329, -1.8142506e-05, -0.0011179153, ..."


In [36]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(English_qoute_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(English_qoute_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + English_qoute_table["output_text"].iloc[index] )  
  print("_"*100) 
     

کارها باید با مشارکت انجام شوند => It's no good pretending that any relationship has a future if your record collections disagree violently or if your favorite films wouldn't even speak to each other if they met at a party
____________________________________________________________________________________________________
نباید غیبت دیگران را بکنیم. => I don't want to be married just to be married. I can't think of anything lonelier than spending the rest of my life with someone I can't talk to, or worse, someone I can't be silent with
____________________________________________________________________________________________________
باید به افرادی که بدهی دارند مهلت بیشتری داد => I finally understood what true love meant...love meant that you care for another person's happiness more than your own, no matter how painful the choices you face might be
____________________________________________________________________________________________________
باید پنهانی کمک‌های خود را به فقرا ب

Nahj Al-Balaqa

In [41]:
columns = ["output_text" , "vector"]
Nahj_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'nahj/Nahj Al-Balaqa.txt', 'r')
Lines = file1.readlines()
nahj = []
count = 1
for line in Lines:
    splited = line.split("\t")
    if len(splited[1]) < 300: 
      Nahj_table = Nahj_table.append({'output_text':splited[1],'vector':laser.embed_sentences(splited[1], 'ar')}, ignore_index = True)
file1.close()


In [42]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(Nahj_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(Nahj_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + Nahj_table["output_text"].iloc[index] )  
  print("_"*100) 
     

کارها باید با مشارکت انجام شوند => وَ قَالَ ع فِی الّذِینَ اعتَزَلُوا القِتَالَ مَعَهُ خَذَلُوا الحَقّ وَ لَم یَنصُرُوا البَاطِلَ

____________________________________________________________________________________________________
نباید غیبت دیگران را بکنیم. => وَ قَالَ ع الفَقِیهُ کُلّ الفَقِیهِ مَن لَم یُقَنّطِ النّاسَ مِن رَحمَةِ اللّهِ وَ لَم یُؤیِسهُم مِن رَوحِ اللّهِ وَ لَم یُؤمِنهُم مِن مَکرِ اللّهِ

____________________________________________________________________________________________________
باید به افرادی که بدهی دارند مهلت بیشتری داد => وَ قَالَ ع لَا یُزَهّدَنّکَ فِی المَعرُوفِ مَن لَا یَشکُرُهُ لَکَ فَقَد یَشکُرُکَ عَلَیهِ مَن لَا یَستَمتِعُ بشِیَ‌ءٍ مِنهُ وَ قَد تُدرِکُ مِن شُکرِ الشّاکِرِ أَکثَرَ مِمّا أَضَاعَ الکَافِرُوَ اللّهُ یُحِبّ المُحسِنِینَ

____________________________________________________________________________________________________
باید پنهانی کمک‌های خود را به فقرا برسانیم => ملَکَتَنیِ عیَنیِ وَ أَنَا جَالِسٌ فَسَنَحَ لِی رَسُولُ اللّهِص فَقُلت

Yek-Beyti

In [46]:
columns = ["output_text" , "vector"]
tak_beit_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'poems/yek-beiti.txt', 'r')
Lines = file1.readlines()
for line in Lines:
    print(line)
    tak_beit_table = tak_beit_table.append({'output_text':line,'vector':laser.embed_sentences(line, 'fa')}, ignore_index = True)
file1.close()
     


در سخن گفتن خطای جاهلان پیدا شود/ تیر کج چون از کمان بیرون دود رسوا شود

ترا دیدم و یوسف را شنیدم/ شنیدن کی بود مانند دیدن

رزق را روزی رسان پر میدهد/ بی مگس هرگز نماند عنکبوت

دل ز پر گفتن بمیرد در بدن/ گر چه گفتارش بود در عدن

امشب تو را به خوبی نسبت به ماه کردم/ تو خوب تر ز ماهی من اشتباه کردم

به همان قدر که چشم تو پر از زیباییست/ بی تو دنیای من ای دوست پر از تنهاییست

در این فکرم که خواهی ماند با من مهربان یا نه/ به من کم میکنی لطفی که داری این زمان یا نه

به صد سال یک دوست آید به دست/ به یک روز توان کرد دشمن به شصت

طاعت آن نیست که بر خاک نهی پیشانی/ صدق پیش آر که ابلیس بسی کرد سجود

ز گفتار بیهوده جز رنج نیست/ چو خاموشی اندر جهان گنج نیست

بزرگی سراسر به گفتار نیست/ دو صد گفته چون نیم کردار نیست

مباش درپی آزار و هرچه خواهی کن/  که در شریعت ما غیر از این گناهی نیست

یک عمر دور و تنها تنها به جرم اینکه/ او سرسپرده میخواست من دل سپرده بودم

قناعت کن اگر در آرزوی گنج قارونی/  گدای خویش باش ار طالب ملک سلیمانی

میرود کز ما جدا گردد ولی/ جان و دل با اوست هر جا میرود

به زیورها بیارای

In [2]:
tak_beit_table

NameError: ignored

*Evaluate* 

In [47]:


for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(tak_beit_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(tak_beit_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + tak_beit_table["output_text"].iloc[index] )  
  print("_"*100) 
     


کارها باید با مشارکت انجام شوند => بلا ندیده دعا را شروع باید کرد/ علاج واقعه پیش از وقوع باید کرد

____________________________________________________________________________________________________
نباید غیبت دیگران را بکنیم. => مباش درپی آزار و هرچه خواهی کن/  که در شریعت ما غیر از این گناهی نیست

____________________________________________________________________________________________________
باید به افرادی که بدهی دارند مهلت بیشتری داد => تواضع گر چه محبوب است و فضل بیکران دارد/ نباید کرد بیش از حد که هیبت را زیان دارد

____________________________________________________________________________________________________
باید پنهانی کمک‌های خود را به فقرا برسانیم => آنجا که جهل مایه تمکین و سروریست/ باید به روز مردم دانا گریستن

____________________________________________________________________________________________________
نباید در خوردن زیاده‌روی کرد => دشمن خود را نباید زد با تبر / گر توانی کشت او را با شکر

________________________________________________________________

غزلیات حافظ







In [49]:


columns = ["output_text" , "vector"]
hafez_table = pd.DataFrame([], columns=columns)
file1 = open( data_set_root +'hafez/hafez-qazals.txt', 'r')
Lines = file1.readlines()
flag = False
qazals = []
for line in Lines:
    # line = line.replace("/","")
    # print(line)
    qazal = r".*غزل.*"
    qazal_reg = f"{qazal}"    
    if(re.match(qazal_reg, line) and len(line)<20):
      print(line)
      input =  '\r\n'.join(qazals)
      # print("_"*100)
      hafez_table = hafez_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa')}, ignore_index = True)
      qazals = []
    else:
      if(len(line)>5): 
        qazals.append(line.strip())

    
file1.close()

     


﻿غزل    ۱

غزل    ۲

غزل    ۳

غزل   ۴

غزل   ۵

غزل    ۶

غزل    ۷

غزل    ۸

غزل    ۹

غزل    ۱۰

غزل    ۱۰

غزل    ۱۲

غزل    ۱۳

غزل    ۱۴

غزل   ۱۵

غزل    ۱۶

غزل    ۱۷

غزل   ۱۸

غزل    ۱۹

غزل    ۲۰

غزل    ۲۱

غزل    ۲۲

غزل    ۲۳

غزل    ۲۴

غزل    ۲۵

غزل    ۲۶

غزل    ۲۷

غزل    ۲۸

غزل    ۲۹

غزل    ۳۰

غزل    ۳۱

غزل    ۳۲

غزل    ۳۳

غزل   ۳۴

غزل    ۳۵

غزل    ۳۶

غزل    ۳۷

غزل    ۳۸

غزل    ۳۹

غزل    ۴۰

غزل    ۴۱

غزل    ۴۲

غزل    ۴۳

غزل    ۴۴

غزل    ۴۵

غزل    ۴۶

غزل    ۴۷

غزل    ۴۸

غزل    ۴۹

غزل    ۵۰

غزل    ۵۱

غزل    ۵۲

غزل    ۵۳

غزل    ۵۴

غزل    ۵۵

غزل    ۵۶

غزل    ۵۷

غزل    ۵۸

غزل    ۵۹

غزل    ۶۰

غزل    ۶۱

غزل    ۶۲

غزل    ۶۳

غزل    ۶۴

غزل    ۶۵

غزل    ۶۶

غزل    ۶۷

غزل    ۶۸

غزل    ۶۹

غزل    ۷۰

غزل    ۷۱

غزل    ۷۲

غزل    ۷۳

غزل    ۷۴

غزل    ۷۵

غزل    ۷۶

غزل    ۷۷

غزل    ۷۸

غزل    ۷۹

غزل    ۸۰

غزل    ۸۱

غزل    ۸۲

غزل    ۸۳

غزل    ۸۴

غزل    ۸۵

غزل    ۸۶

غزل    ۸۷

غزل    ۸۸

غزل    ۸۹

غزل    ۹۰

غزل    ۹۱

غزل    ۹۲

غ

In [50]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(hafez_table)-1):
    image = np.array(input_table["vector"].iloc[k])@np.array(hafez_table["vector"].iloc[i+1]).T
    if(image > max):
      max = image
      index = i+1

  print(input_table["input_text"].iloc[k] + "  ======>>>> " + " غزل شماره " + str(index))  
  print(hafez_table["output_text"].iloc[index] )
  print("_"*100) 

کارها باید با مشارکت انجام شوند  ======>>>>  غزل شماره 244
معاشران گره از زلف يار باز کنيد
شبی خوش است بدين قصه‌اش دراز کنيد
حضور خلوت انس است و دوستان جمعند
و ان يکاد بخوانيد و در فراز کنيد
رباب و چنگ به بانگ بلند می‌گويند
که گوش هوش به پيغام اهل راز کنيد
به جان دوست که غم پرده بر شما ندرد
گر اعتماد بر الطاف کارساز کنيد
ميان عاشق و معشوق فرق بسيار است
چو يار ناز نمايد شما نياز کنيد
نخست موعظه پير صحبت اين حرف است
که از مصاحب ناجنس احتراز کنيد
هر آن کسی که در اين حلقه نيست زنده به عشق
بر او نمرده به فتوای من نماز کنيد
وگر طلب کند انعامی از شما حافظ
حوالتش به لب يار دلنواز کنيد
____________________________________________________________________________________________________
نباید غیبت دیگران را بکنیم.  ======>>>>  غزل شماره 378
ما نگوييم بد و ميل به ناحق نکنيم
جامه کس سيه و دلق خود ازرق نکنيم
عيب درويش و توانگر به کم و بيش بد است
کار بد مصلحت آن است که مطلق نکنيم
رقم مغلطه بر دفتر دانش نزنيم
سر حق بر ورق شعبده ملحق نکنيم
شاه اگر جرعه رندان نه به حرمت نوشد
التفاتش به می صاف مروق نکنيم

غزلیات شمس

In [64]:
moulavi_qazals_path = data_set_root + "moulavi/divane-e-shams.txt"
moulavi_qazals_pattern = re.compile(r'\n?\s*\d+\s*?\n')

def read_divan_shams():
    with open(moulavi_qazals_path, encoding='utf-8') as f:
        text = f.read()
        
    qazals_arr = moulavi_qazals_pattern.split(text)
    return qazals_arr[1:]


In [66]:
columns = ["output_text" , "vector"]
molavi_table = pd.DataFrame([], columns=columns)

for i in range(0, 500):
  input = read_divan_shams()[i][0:500]
  print(i)
  molavi_table = molavi_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa')}, ignore_index = True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [67]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(molavi_table)-1):
    image = np.array(input_table["vector"].iloc[k])@np.array(molavi_table["vector"].iloc[i+1]).T
    if(image > max):
      max = image
      index = i+1

  print(input_table["input_text"].iloc[k] + "  ======>>>> ")  
  print(molavi_table["output_text"].iloc[index] )
  print("_"*100) 

کارها باید با مشارکت انجام شوند  ======>>>> 
امتزاج روح ها در وقت صلح و جنگ ها	با کسی باید که روحش هست صافی صفا
چون تغییر هست در جان وقت جنگ و آشتی	آن نه یک روحست تنها بلک گشتستند جدا
چون بخواهد دل سلام آن یکی همچون عروس	مر زفاف صحبت داماد دشمن روی را

باز چون میلی بود سویی بدان ماند که او	میل دارد سوی داماد لطیف دلربا
از نظرها امتزاج و از سخن ها امتزاج	وز حکایت امتزاج و از فکر آمیزها
همچنانک امتزاج ظاهرست اندر رکوع	وز تصافح وز عناق و قبله و مدح و دعا
بر تفاوت این تمازج ها ز میل و نیم میل	وز سر کره و کراهت وز پی ترس و حیا
آن رکوع باتانی وا
____________________________________________________________________________________________________
نباید غیبت دیگران را بکنیم.  ======>>>> 
زان شاه که او را هوس طبل و علم نیست	دیوانه شدم بر سر دیوانه قلم نیست
از دور ببینی تو مرا شخص رونده	آن شخص خیالست ولی غیر عدم نیست
پیش آ و عدم شو که عدم معدن جانست	اما نه چنین جان که بجز غصه و غم نیست
من بی من و تو بی تو درآییم در این جو	زیرا که در این خشک بجز ظلم و ستم نیست
این جوی کند غرقه ولیکن نکشد مرد	کو آب